In [1]:
# requirements

# !pip install requests
# !pip install time
# !pip install pandas
# !pip install numpy
# !pip install matplotlib.pyplot
# !pip install schedule
!pip install plotly




# For Reviewers:
The purpose of our tutorial is to show how to make data-informed investment decisions through visualizing and analyzing past trends in cryptocurrencies and index fund(s). Our target users are beginner investors who are interested in crypto markets but have limited coding experience. We assume that the user will have a basic understanding of Python but limited exposure to data science. Our tutorial will address user needs like having accessibility of financial data analysis and practical decision making tools. Beginners will struggle to gather, clean, and visualize data so we will be providing step-by-step instructions for doing so. Also through seeing risk and reward tradeoffs and making a predictive AI model, users will gain skills in comparing assets and products and be able to experiment with financial strategies like holding and buying. Our tutorial will focus on data based coding while ensuring that users learn the concepts of financial data analysis and the workflows necessary to implement and visualize them.

The priorties for things to review at the high level would be our graphs. Our priorities is to make the graph(s) as clear as possible and the data cleaning step. We are also not sure if the numpy and pandas methods that we use are too advanced for our target audience.

Whats Done: We implemented getting the cryptocurrency data from an API, cleaning the data, calculaing the risk/reward from that data, graphing the data, and scheduling it so that the pipeline runs everyday at 9 am.

Whats missing/left to do is that we wanted to add an interactive element to the graph so that you can over specific dates on the graph and be able to see that days closing price/violatility. Additionally we still need write the call for an OpenAI LLM to give advice based on the prices/violatility. Finally, we still need to change the code so that multiple/different coins can be used instead of just bitcoin.

Additional Notes: All of the requirement libraries are listed above. For the scheduling code, you can change 09:00 to next closest minute in order to test that segment of code. 



# Section: Calling CoinGecko's API

An Application Programming Interface (API) is basically a set of rules of how other people can interact with a service. In this tutorial, we are going to request historial data on different cryptocurrineces using Python's request library. 


# Step 1 - API URL

CoinGecko documentation: https://docs.coingecko.com/reference/coins-id-market-chart-range

The request library in Python sends a request to an URL (API Endpoint) for its data. Based on the documentation, an endpoint that allows us to get historical data for a certain coin is at this URL: "https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"

CoinGecko coin id list: https://docs.google.com/spreadsheets/d/1wTTuxXt8n9q7C4NDXqQpI3wpKu1_5bGVmP9Xz0XGSyU/edit?gid=0#gid=0

Looking at the URL, it contains the parameter: {coin_id}. Coin_id is the name of the cryptocurrency that we want the historical data for. For this example lets use bitcoin



In [2]:
import requests 

coin_id = "bitcoin"

url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"

response = requests.get(url)

data = response.json()

print(data)

{'status': {'timestamp': '2025-12-08T00:29:44.846Z', 'error_code': 10014, 'error_message': "Invalid value for 'to' and 'from' provided for the date parameter. Use YYYY-MM-DD, YYYY-MM-DDTHH:MM or UNIX timestamp only."}}


# Step 2 - Query Parameters

After running the code cell above, you might have noticed that the request errored because there was an invalid value for "to" and "from". This is because on top of giving the API Endpoint a coin_id, we also need to input what currency we want the data in and from what time range. These required parameters can be found in the documentation. 

<img src="api_param_img.png" alt="title" width="50%">

For the currency, we can use the example that they give us and input it as USD. The documentation states that the date ranges can be a UNIX timestamp. We want the most recent 3 months of data. To get the most recent 3 months as a UNIX timestamp we can use the time python library. time.time() returns the current time as a UNIX timestamp. Then we can calculate the number of seconds in 3 months (90 * 24 * 60 * 60) and subtract that from the current time to get the UNIX timestamp of 3 months ago from right now



In [11]:
import requests
import time

coin_id = "bitcoin"

url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"

to_ts = int(time.time())                 
from_ts = to_ts - 90 * 24 * 60 * 60    

params = {    
    "vs_currency": "usd",
    "from": from_ts,
    "to": to_ts        
}

response = requests.get(url, params=params)
data = response.json()
first_ten = {data_type: data[data_type][:10] for data_type in data}

print(first_ten)

{'prices': [[1757379745940, 111652.0713866927], [1757383232178, 111407.09074159958], [1757386969021, 111287.14885295804], [1757390559597, 111702.97471470888], [1757394146616, 112028.60805847727], [1757397716217, 112140.77309366502], [1757401378293, 113112.94201935548], [1757404923129, 113070.31854691613], [1757408595191, 112951.8697006361], [1757412173726, 112921.314592305]], 'market_caps': [[1757379745940, 2223921193215.7153], [1757383232178, 2219402478359.0503], [1757386969021, 2216872957725.678], [1757390559597, 2224210973700.837], [1757394146616, 2227896450626.2056], [1757397716217, 2232649720471.2764], [1757401378293, 2250069689814.452], [1757404923129, 2250861688201.6816], [1757408595191, 2249827834806.4517], [1757412173726, 2249389015482.738]], 'total_volumes': [[1757379745940, 37397075617.940674], [1757383232178, 37649577071.57709], [1757386969021, 38290383173.37138], [1757390559597, 37921869431.4049], [1757394146616, 38533245274.096085], [1757397716217, 39136969373.14936], [17

# Step 3 - Multiple Coins

In order to see the historical market data for different coins, all we have to change is the coin id based on the list linked above. For this tutorial we will also look at etherum and dogecoin in addition to bitcoin. In order to avoid repeated code, we wrapped the code in a function with an argument for the coin id.

In [15]:
import requests
import time

def fetch_data(coin_id):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"

    to_ts = int(time.time())                 
    from_ts = to_ts - 90 * 24 * 60 * 60    

    params = {    
        "vs_currency": "usd",
        "from": from_ts,
        "to": to_ts        
    }

    response = requests.get(url, params=params)
    data = response.json()

    return data

btc_data = fetch_data("bitcoin")
eth_data = fetch_data("ethereum")
doge_data = fetch_data("dogecoin")

btc_first_ten = {data_type: btc_data[data_type][:10] for data_type in btc_data}
eth_first_ten = {data_type: eth_data[data_type][:10] for data_type in eth_data}
doge_first_ten = {data_type: doge_data[data_type][:10] for data_type in doge_data}

print(btc_first_ten)
print(eth_first_ten)
print(doge_first_ten)


{'prices': [[1757379745940, 111652.0713866927], [1757383232178, 111407.09074159958], [1757386969021, 111287.14885295804], [1757390559597, 111702.97471470888], [1757394146616, 112028.60805847727], [1757397716217, 112140.77309366502], [1757401378293, 113112.94201935548], [1757404923129, 113070.31854691613], [1757408595191, 112951.8697006361], [1757412173726, 112921.314592305]], 'market_caps': [[1757379745940, 2223921193215.7153], [1757383232178, 2219402478359.0503], [1757386969021, 2216872957725.678], [1757390559597, 2224210973700.837], [1757394146616, 2227896450626.2056], [1757397716217, 2232649720471.2764], [1757401378293, 2250069689814.452], [1757404923129, 2250861688201.6816], [1757408595191, 2249827834806.4517], [1757412173726, 2249389015482.738]], 'total_volumes': [[1757379745940, 37397075617.940674], [1757383232178, 37649577071.57709], [1757386969021, 38290383173.37138], [1757390559597, 37921869431.4049], [1757394146616, 38533245274.096085], [1757397716217, 39136969373.14936], [17

# Section: Cleaning and Structuring Data
In the previous section, we wrote a fetch_data(coin_id) function that calls the CoinGecko API and returns raw historical data for a cryptocurrency.

The good news: this raw data already has a lot of useful information.
The bad news: it’s not in a form we can easily analyze.

In this section, we’ll turn that raw JSON into a clean, daily table with: 
- Open, High, Low, Close price (OHLC)
- Daily trading volume 
- Daily market cap

We’ll do this for multiple coins, so later we can compare them.





# Step 1 - Inspecting What the API Actually Returns
Before we write any cleaning code, it’s important to look at the shape of the raw data. This is good practice anytime you work with an API.

In [16]:
sample = fetch_data("bitcoin")
print(sample.keys())

dict_keys(['prices', 'market_caps', 'total_volumes'])


You should see something like: dict_keys(['prices', 'market_caps', 'total_volumes'])

So the API is giving us three separate time series:
- prices — historical prices
- market_caps — historical market capitalization
- total_volumes — historical trading volume

Each of these is a list of pairs:

In [17]:
for key in sample.keys():
    print(f"\n--- {key} ---")
    print(sample[key][:3])  # look at first 3 entries


--- prices ---
[[1757379745940, 111652.0713866927], [1757383232178, 111407.09074159958], [1757386969021, 111287.14885295804]]

--- market_caps ---
[[1757379745940, 2223921193215.7153], [1757383232178, 2219402478359.0503], [1757386969021, 2216872957725.678]]

--- total_volumes ---
[[1757379745940, 37397075617.940674], [1757383232178, 37649577071.57709], [1757386969021, 38290383173.37138]]


All three have the same basic format:
- The first element: a timestamp in milliseconds since 1970 (called Unix time)
- The second element: the value at that time

This is good news: it means we can process all three with the same helper function.
# Step 2 - Converting Raw Lists into a Clean Time Series
Right now, each series is just a Python list of lists. We want to turn it into a pandas DataFrame that has a proper datetime column, a clearly named value column, is sorted in chronological order, and handles non-numeric and obvious bad values.

We can write one helper function that does this:

In [18]:
import pandas as pd
import numpy as np

def _to_df(series, value_name):
    """
    Convert CoinGecko lists like [[ts_ms, value], ...] into a clean pandas DataFrame.

    - ts_ms: timestamp in milliseconds since 1970 (Unix time)
    - value_name: the name we want for the numeric column (e.g., "price", "mcap")

    Returns a DataFrame with:
    - 'date_utc' column as a proper UTC datetime
    - one numeric column with the given value_name
    - rows sorted chronologically
    """
    # Turn raw list into a table with two columns: ts_ms and the value
    df = pd.DataFrame(series, columns=["ts_ms", value_name])

    # Convert timestamp from milliseconds → actual datetime in UTC
    df["date_utc"] = pd.to_datetime(df["ts_ms"], unit="ms", utc=True)

    # Drop the raw timestamp column now that we have a readable datetime
    df = df.drop(columns=["ts_ms"]).sort_values("date_utc")

    # Ensure the value column is numeric, and treat 0 as missing (often signals bad data)
    df[value_name] = (
        pd.to_numeric(df[value_name], errors="coerce")
          .replace(0, np.nan)
    )

    return df


You can test this on each of the three series and see that we have clean time series with real datetimes instead of unreadable integers.

In [19]:
prices_df    = _to_df(sample["prices"], "price")
mcap_df      = _to_df(sample["market_caps"], "mcap")
volumes_df   = _to_df(sample["total_volumes"], "vol_ccy")  # volume in currency (USD)

prices_df.head()

,price,date_utc
0,111652.071387,2025-09-09 01:02:25.940000+00:00
1,111407.090742,2025-09-09 02:00:32.178000+00:00
2,111287.148853,2025-09-09 03:02:49.021000+00:00
3,111702.974715,2025-09-09 04:02:39.597000+00:00
4,112028.608058,2025-09-09 05:02:26.616000+00:00


# Step 3 - Aligning Prices, Market Cap, and Volume in Time
Each series (price, market cap, volume) is recorded independently, and their timestamps might not line up perfectly. For example we might have:
- A price at 10:00:00.123
- A market cap at 10:00:05.456
- A volume at 10:01:01.789

What we want though is one combined table where each row has price, market cap, and volume at approximately the same time.

To do this, we use pandas.merge_asof(), which is like a time-aware join: it matches each row with the closest timestamp from the other table.

In [20]:
def align_all_series(data_json):
    """
    Take the raw JSON from CoinGecko and align
    price, market cap, and volume on a common time axis.
    """
    prices_df  = _to_df(data_json["prices"], "price")
    mcap_df    = _to_df(data_json["market_caps"], "mcap")
    volumes_df = _to_df(data_json["total_volumes"], "vol_ccy")

    # First merge: prices with market caps (nearest timestamp)
    merged = pd.merge_asof(
        prices_df.sort_values("date_utc"),
        mcap_df.sort_values("date_utc"),
        on="date_utc",
        direction="nearest",
        tolerance=pd.Timedelta("5min"),
    )

    # Second merge: bring in volumes the same way
    merged = pd.merge_asof(
        merged.sort_values("date_utc"),
        volumes_df.sort_values("date_utc"),
        on="date_utc",
        direction="nearest",
        tolerance=pd.Timedelta("5min"),
    )

    # Small cleanup: fill tiny gaps and drop rows that are still missing anything important
    merged = merged.ffill(limit=3).dropna(subset=["price", "mcap", "vol_ccy"])

    # Use datetime as index for resampling later
    merged = merged.set_index("date_utc").sort_index()

    return merged


What this is doing:
- direction="nearest": match the closest measurement in time
- tolerance=5min: don’t match points that are more than 5 minutes apart
- ffill(limit=3): if there’s a short gap (like one missing reading), copy the last known value forward
- .dropna(...): remove any remaining rows where something critical is missing

Now we have a high-frequency time series where each row has price, market cap, and volume.
We can test and see this:

In [21]:
aligned_tick = align_all_series(sample)
aligned_tick.head()

,price,mcap,vol_ccy
date_utc,,,
2025-09-09 01:02:25.940000+00:00,111652.071387,2.223921e+12,3.739708e+10
2025-09-09 02:00:32.178000+00:00,111407.090742,2.219402e+12,3.764958e+10
2025-09-09 03:02:49.021000+00:00,111287.148853,2.216873e+12,3.829038e+10
2025-09-09 04:02:39.597000+00:00,111702.974715,2.224211e+12,3.792187e+10
2025-09-09 05:02:26.616000+00:00,112028.608058,2.227896e+12,3.853325e+10


# Step 4 - Resampling to Daily OHLCV
Right now, aligned_tick may have data points every few minutes. For analysis, though, we  want one row per day with summary statistics.

Some typical finance key terms and indicators in the market that we will use are
- Open — first price of the day
- High — max price of the day
- Low — min price of the day
- Close — last price of the day
- Volume — total traded volume during that day

We’ll use pandas’ .resample("1D") to group by day and then apply aggregations.

In [22]:
def coingecko_to_daily_table(data_json):
    """
    Convert CoinGecko market_chart/range JSON into a daily table with:
    - Open, High, Low, Close prices
    - Daily Market Cap (last value of each day)
    - Daily Volume in quote currency (sum of intraday volume)
    """
    tick = align_all_series(data_json)  # high-frequency aligned data

    # Daily OHLC for price
    ohlc = tick["price"].resample("1D").agg(
        Open="first",
        High="max",
        Low="min",
        Close="last"
    )

    # Daily market cap: we take the last value in each day (end-of-day cap)
    daily_mcap = tick["mcap"].resample("1D").agg(
        Market_Cap="last"
    )

    # Daily volume in currency (e.g., USD): sum of intraday volumes
    daily_vol = tick["vol_ccy"].resample("1D").agg(
        Volume_Currency="sum"
    )

    # Combine everything into one table
    daily = pd.concat([ohlc, daily_mcap, daily_vol], axis=1)
    daily.index.name = "Date"

    # Drop days where everything is missing
    daily = daily.dropna(how="all")

    return daily


# Step 5 - Doing This for Multiple Coins
Now that we have a function that is reusable, we can apply it to multiple cryptocurrencies that we want to analyze by just using a loop.

In [23]:
coins = ["bitcoin", "ethereum", "dogecoin"]

daily_data = {}
for coin in coins:
    raw = fetch_data(coin)
    daily = coingecko_to_daily_table(raw)
    daily_data[coin] = daily

daily_data["ethereum"].head()

,Open,High,Low,Close,Market_Cap,Volume_Currency
Date,,,,,,
2025-09-09 00:00:00+00:00,4301.791442,4370.902388,4286.163928,4316.538729,5.209414e+11,6.712393e+11
2025-09-10 00:00:00+00:00,4311.858429,4415.422957,4286.931501,4347.328637,5.247430e+11,7.017385e+11
2025-09-11 00:00:00+00:00,4347.482124,4459.155486,4347.482124,4459.155486,5.381575e+11,8.461871e+11
2025-09-12 00:00:00+00:00,4459.319016,4703.275010,4459.319016,4703.275010,5.670344e+11,8.431016e+11
2025-09-13 00:00:00+00:00,4708.841744,4762.735941,4620.425114,4660.790981,5.625633e+11,8.964197e+11


Now daily_data is a dictionary where:
- daily_data["bitcoin"] is BTC’s daily OHLC + market cap + volume
- daily_data["ethereum"] is ETH’s daily table
- daily_data["dogecoin"] is DOGE’s table

All have the same strucutre and are now ready for comparisons and analysis. 

In [24]:
import plotly.graph_objects as go

btc_data = daily_data["bitcoin"]

dates = btc_data.index.to_list()

close_prices = btc_data["Close"].to_list()

def plot_close_prices(close_prices, dates):

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=dates,
            y=close_prices,
            mode="lines+markers",      # line + scatter points
            name="BTC Close Price"
        )
    )

    fig.update_layout(
        title="Bitcoin Daily Close Price",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        hovermode="x unified"
    )

    fig.show()

plot_close_prices(close_prices, dates)

In [26]:
import plotly.graph_objects as go

coins = ["bitcoin", "ethereum", "dogecoin"]

fig = go.Figure()

for coin in coins:
    df = daily_data[coin]        
    dates = df.index.to_list() 
    closes = df["Close"].to_list()

    fig.add_trace(
        go.Scatter(
            x=dates,
            y=closes,
            mode="lines+markers",
            name=f"{coin.capitalize()}"
        )
    )

fig.update_layout(
    title="Daily Close Prices for Multiple Cryptocurrencies",
    xaxis_title="Date",
    yaxis_title="Price (USD)",
    hovermode="x unified"
)

fig.show()

# Step 6 - Analyzing Risk/Return of a Coin


Looking at the risk/return ratio of a coin helps us make a more informed buying decision.


Our risk/return function uses the Sharpe Ratio, which is a widely-accepted formula for evaluating the risk of buying a cryptocurrency. The function takes in an array of prices and a risk-free return value. The risk free-rate (rf) is the percentage return you can earn by investing in a Treasury Bill instead of the cryptocurrency being analyzed. As of November 16, 2025, the rf is 3.89%. You can find this value by going to the US department of treasury website.


In our function body, we first calculate the daily returns and store these values in an array called returns. With that, we can calculate the risk by taking the standard deviation of the returns array and calculate the reward by taking the mean of the returns array. The final risk/reward ratio is calculated by subtracting the risk-free rate from the reward, then dividing that value by the risk.


In [27]:
import numpy as np
import math

# Sharp Ratio Grading Thresholds:
# Less than 0: high risk/low reward
# 0.0 - 0.99: low risk/low reward
# 1.0 – 1.99: Adequate/good
# Greater than 2: Very high reward potential

# prices: array of prices
# rf: risk-free rate of return
def risk_return(prices, rf=0.04):
    returns = [((prices[i] - prices[i-1]) / prices[i-1]) for i in range(1, len(prices))]
    risk = np.std(returns)
    reward = np.mean(returns)
    return np.round((reward - rf / risk), 2)

Lets look at the risk/reward ratio for bitcoin. Lets grab bitcoin's 90 day closing price data from the daily_data data frame and call our risk_return function on the column 'Close', which contains the closing prices.

In [28]:
btc_closing_prices = daily_data['bitcoin']['Close']
print(risk_return(btc_closing_prices))

-1.83


/var/folders/d4/h231xr_d1k71q58z7dpq7jd00000gn/T/ipykernel_26669/3999890648.py:13: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



A negative risk/reward value indicates this investment should be avoided because it is very risky and has low reward potential. Risk/reward ratios in the 0.0-0.99 range indicate a safe investment. Most investments fall in the 1.0-1.99 range and these offer a balance between risk and reward. A ratio above 2 suggests use of leverage to inflate returns, which also increases the risk. Thus, it is encouraged that investors conduct further research on such investments.


# Step 7: Volatility


The volatility function shows how frequently a cryptocurrency price moves. Our function takes in an array of prices as input and an integer n that represents the number of days we want to check volatility for. To calculate daily volatility, we will use a 30-day window of past prices, which means the value of n cannot exceed 60, since we have data for 90 days.

In our volatility function, we will again first calculate the returns (like we did for the risk/return function). We also calculate the average of the returns. We then create an empty array called crypto_volatility, where we will store our daily volatility values. We will create a nested for loop. Each outer loop iteration represents the volatility for one day. The inner loop calculates the volatility for that day using the daily price values from the past 30 days. The volatility for that day is then calculated by using the standard deviation formula. We convert the volatility to a percentage value by multiplying by 100 and make it more readable by using python's round() function. We then store the daily volatility value in our crypto_volatility array.


In [29]:
# n represents number of days. The value of n cannot be above 60 since we are using a 30-day window to calculate the volatility and we have data for 90 days.
def volatility(prices, n, window=30):
    returns = [((prices[i] - prices[i-1]) / prices[i-1]) for i in range(1, len(prices))]
    crypto_volatility = []
    for i in range(90-n, 90):
        window_returns = returns[i-window:i]
        average = np.mean(window_returns)
        ssd = 0
        for r in window_returns:
            ssd += (r - average)**2
        daily_volatility = round(math.sqrt(ssd / window) * 100, 2)
        crypto_volatility.append(daily_volatility)
    return crypto_volatility



Lets now look at volatility for bitcoin in the past 60 days. This time, lets use the opening prices.

In [30]:
btc_opening_prices = daily_data['bitcoin']['Open']
btc_volatility = volatility(btc_opening_prices, 60, 30)
print(btc_volatility)

[1.53, 1.55, 2.01, 2.03, 2.16, 2.16, 2.18, 2.22, 2.23, 2.25, 2.25, 2.26, 2.29, 2.31, 2.28, 2.32, 2.31, 2.21, 2.25, 2.25, 2.22, 2.24, 2.26, 2.15, 2.12, 2.1, 2.19, 2.3, 2.35, 2.34, 2.36, 2.36, 2.08, 2.06, 1.96, 1.96, 1.97, 2.12, 2.11, 2.11, 2.13, 2.12, 2.08, 2.23, 2.24, 2.17, 2.23, 2.26, 2.17, 2.32, 2.34, 2.31, 2.3, 2.28, 2.37, 2.65, 2.63, 2.51, 2.51, 2.48]


/var/folders/d4/h231xr_d1k71q58z7dpq7jd00000gn/T/ipykernel_26669/1872290232.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



Lets also look at the average volatility.

In [31]:
avg_volatility = np.round(np.mean(btc_volatility), 2)
print(f'Average volatility over 60 days: {avg_volatility}')

Average volatility over 60 days: 2.21


# Step 8: Plotting Volatility


Next, we will use Plotly to plot bitcoin's volatility in the past 60 days. The x-axis represents dates and the y-axis represents volatility (in percentage). The graph takes advantage of Plotly's hover functionality to allow users to see specific volatility values for specific dates.


btc_opening_prices = daily_data['bitcoin']['Open']
btc_volatility = volatility(btc_opening_prices, 60)
avg_volatility = np.round(np.mean(btc_volatility), 2)
print(btc_volatility)
print(f'Average volatility over 60 days: {avg_volatility}')

In [33]:
import plotly.graph_objects as go

number_of_days = 60

btc_opening_prices = daily_data['bitcoin']['Open']

dates = btc_opening_prices.tail(number_of_days).index.to_list()

btc_volatility = volatility(btc_opening_prices, number_of_days, 30)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=dates,
        y=btc_volatility,
        mode="lines+markers",      # line + scatter points
        name="BTC Volatility"
    )
)

fig.update_layout(
    title="Bitcoin Volatility",
    xaxis_title="Date",
    yaxis_title="Volatility",
    hovermode="x unified"
)

fig.show()



/var/folders/d4/h231xr_d1k71q58z7dpq7jd00000gn/T/ipykernel_26669/1872290232.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



We can make multiple plots in the same graph to compare the volatilities of different cryptocurrencies.


In [35]:
import plotly.graph_objects as go

coins = ["bitcoin", "ethereum", "dogecoin"]


number_of_days = 60

fig = go.Figure()

for coin in coins:
    btc_opening_prices = daily_data[coin]['Open']       
    dates = btc_opening_prices.tail(number_of_days).index.to_list()
    btc_volatility = volatility(btc_opening_prices, number_of_days, 30)

    fig.add_trace(
        go.Scatter(
            x=dates,
            y=btc_volatility,
            mode="lines+markers",
            name=f"{coin.capitalize()}"
        )
    )

fig.update_layout(
    title="Daily Volatility for Multiple Cryptocurrencies",
    xaxis_title="Date",
    yaxis_title="Volatility",
    hovermode="x unified"
)

fig.show()

/var/folders/d4/h231xr_d1k71q58z7dpq7jd00000gn/T/ipykernel_26669/1872290232.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



When investing in cryptocurrency, it is important to have a diverse portfolio to minimize risk of losing everything in one investment. Here, Plotly's hover functionality is particularly useful as it gives us information we can use to decide how much to allocate to each cryptocurrency. Advanced traders can profit off of high volatility but for beginners, it is best to allocate most of one's portfolio to cryptocurrencies with lower volatility.


# Step 9: Buy or Sell Quiz


Mock trades are a good way to practice making buy/sell decisions and learn how to swing trade, which are trades that profit off of short-medium term investments that last a few days to a few weeks. In our game, the user is given a specific day to either buy or sell a cryptocurrency. The goal is to use the tools we've given you: a price chart, risk/return, and volatility, to make the buy/sell decision.


In [37]:
def game(coin_data):
    btc_data = coin_data["bitcoin"]
    dates = btc_data.index.to_list()
    prices = btc_data["Close"].to_list()
    mid = len(prices) // 2
    plot_close_prices(prices[:mid+1], dates[:mid+1])
    answer = ''
    profit = prices[len(prices)-1]-prices[mid]
    if profit > 0:
        answer = 'buy'
    elif profit < 0:
        answer = 'sell'
    guess = input("Buy or sell? ")
    plot_close_prices(prices[mid:], dates[mid:])
    if answer == '':
        print("No profit")
    elif guess.lower() == answer:
        print("Correct!")
    else:
        print("Incorrect.")

game(daily_data)

Correct!
